In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [9]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_101556\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_23040s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [8]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-427.401395,-83.874285,0,2,1994.550491,3813.550491,0.497444,0.497444,0,-427.401395,None,21.669561,-890.392278,4.546983,-73.370069,0.508633
1,2,1,1,NaN,-635.447272,-82.187646,0,3,2046.357913,3811.291415,0.338977,0.321320,1,-635.447272,"-1097,-1095",-4.455403,-889.221020,-0.923948,-73.185057,0.762949
2,3,1,1,NaN,-401.259592,-74.707427,0,2,2042.486745,3810.000000,0.499824,0.000086,1,-401.259592,-1095,-2.502384,-888.572155,-0.515724,-73.077809,0.508633
3,4,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1099,105.836138,-888.242242,21.967109,-72.768028,0.762949
4,5,1,1,NaN,-412.025671,-77.163422,0,2,1877.470658,3809.470658,0.499138,0.499138,1,-412.025671,"-1098,9854",80.715163,-888.407168,16.705742,-72.902785,0.508633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1258562,1351962,95,8,NaN,62.889782,30.388510,0,2,185.000000,345.555223,0.000000,0.496941,1,NaN,"1191602,1205510",-289.366323,364.696826,-18.979317,20.815305,32.633409
1258563,1351963,95,8,NaN,98.754549,33.212464,0,3,227.701302,353.672805,0.323634,0.331767,1,NaN,"1218815,1286465",-116.878935,397.486941,-7.658059,22.882639,48.950114
1258564,1351964,95,8,NaN,99.933008,33.264018,0,3,205.380426,357.309301,0.343294,0.326829,1,NaN,1258662,-207.041749,412.176181,-13.763066,23.899110,48.950114
1258565,1351965,95,8,NaN,57.859620,24.579948,0,2,315.496376,367.496376,0.499987,0.499987,0,NaN,None,237.759967,453.325791,16.239208,26.705742,32.633409


### Calculate lifetimes

In [10]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 978346/978346 [2:47:21<00:00, 97.43it/s]  


### Drop single frame objects and change frame to time

In [11]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_23040s.pnds')